# Face Recognition Scanner

This notebook performs face recognition using template matching and PCA features with cosine_similarity checking.

In [1]:
import cv2
import json
import os
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

In [15]:
class FaceScanner:
    def __init__(self, model_path, detection_json_path):
        """
        Initialize face scanner
        
        Args:
            model_path: Path to trained PCA model
            detection_json_path: Path to JSON file generated by detection-v2.py
        """
        self.model_path = model_path
        self.detection_json_path = detection_json_path
        self.model_data = None
        self.detection_data = None
        self.template_image = None
        self.is_loaded = False
        
    def load_model_and_data(self):
        """
        Load PCA model and detection data
        """
        # Load PCA model
        if not os.path.exists(self.model_path):
            print(f"Error: Model file {self.model_path} not found!")
            return False
        
        with open(self.model_path, 'rb') as f:
            self.model_data = pickle.load(f)
        
        print(f"PCA model loaded: {len(self.model_data['face_features'])} faces, {len(self.model_data['person_id_map'])} persons")
        
        # Load detection data
        if not os.path.exists(self.detection_json_path):
            print(f"Error: Detection JSON {self.detection_json_path} not found!")
            return False
        
        with open(self.detection_json_path, 'r', encoding='utf-8') as f:
            self.detection_data = json.load(f)
        
        print(f"Detection data loaded: {len(self.detection_data['faces'])} detected faces")
        
        # Load first face image as template
        if self.detection_data['faces']:
            first_face = self.detection_data['faces'][0]
            template_path = first_face['image_path']
            if os.path.exists(template_path):
                self.template_image = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
                print(f"Template image loaded: {template_path}")
            else:
                print(f"Warning: Template image not found: {template_path}")
        
        self.is_loaded = True
        return True
    
    def get_reference_positions(self, frame_number, tolerance=5):
        """
        Get reference positions around specified frame
        
        Args:
            frame_number: Current frame number
            tolerance: Frame number tolerance range
        
        Returns:
            list: List of reference positions
        """
        if not self.is_loaded:
            return []
        
        reference_positions = []
        
        for face_data in self.detection_data['faces']:
            face_frame = face_data['frame_number']
            
            # Check if frame number is within tolerance range
            if abs(face_frame - frame_number) <= tolerance:
                reference_positions.append({
                    'x': face_data['x'],
                    'y': face_data['y'],
                    'width': face_data['width'],
                    'height': face_data['height'],
                    'center_x': face_data['center_x'],
                    'center_y': face_data['center_y'],
                    'frame_diff': abs(face_frame - frame_number)
                })
        
        # Sort by frame difference, prioritize closest frames
        reference_positions.sort(key=lambda x: x['frame_diff'])
        
        return reference_positions
    
    def template_match_around_position(self, frame, template, ref_pos, search_scale=1.5):
        """
        Perform template matching around reference position
        
        Args:
            frame: Current frame
            template: Template image
            ref_pos: Reference position
            search_scale: Search area expansion factor
        
        Returns:
            tuple: (match position, match score)
        """
        h, w = frame.shape[:2]
        th, tw = template.shape[:2]
        
        # Calculate search area
        search_w = int(ref_pos['width'] * search_scale)
        search_h = int(ref_pos['height'] * search_scale)
        
        # Determine search area boundaries
        start_x = max(0, ref_pos['center_x'] - search_w // 2)
        end_x = min(w, ref_pos['center_x'] + search_w // 2)
        start_y = max(0, ref_pos['center_y'] - search_h // 2)
        end_y = min(h, ref_pos['center_y'] + search_h // 2)
        
        # Extract search region
        search_region = frame[start_y:end_y, start_x:end_x]
        
        if search_region.shape[0] < th or search_region.shape[1] < tw:
            return None, 0.0
        
        # Template matching
        result = cv2.matchTemplate(search_region, template, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(result)
        
        # Convert to global coordinates
        global_x = start_x + max_loc[0]
        global_y = start_y + max_loc[1]
        
        return (global_x, global_y, tw, th), max_val
    
    def extract_face_features(self, face_img):
        """
        Extract face features
        
        Args:
            face_img: Face image
        
        Returns:
            numpy.ndarray: PCA feature vector
        """
        if not self.is_loaded:
            return None
        
        # Preprocessing: convert to grayscale and resize
        if len(face_img.shape) == 3:
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        else:
            gray = face_img
        
        resized = cv2.resize(gray, (64, 64))
        flattened = resized.flatten().reshape(1, -1)
        
        # Normalization and PCA transformation
        scaled = self.model_data['scaler'].transform(flattened)
        features = self.model_data['pca'].transform(scaled)
        
        return features[0]
    
    def recognize_face(self, face_features, threshold=0.7):
        """
        Recognize face
        
        Args:
            face_features: Face feature vector
            threshold: Similarity threshold
        
        Returns:
            tuple: (person_id, person_name, confidence)
        """
        if not self.is_loaded or face_features is None:
            return -1, "unknown", 0.0
        
        # Calculate similarity with all known faces
        similarities = cosine_similarity([face_features], self.model_data['face_features'])[0]
        
        # Find highest similarity
        max_idx = np.argmax(similarities)
        max_similarity = similarities[max_idx]
        
        if max_similarity >= threshold:
            person_id = self.model_data['face_labels'][max_idx]
            # Find person_name based on person_id
            person_name = "unknown"
            for name, pid in self.model_data['person_id_map'].items():
                if pid == person_id:
                    person_name = name
                    break
            return person_id, person_name, max_similarity
        else:
            return -1, "unknown", max_similarity

    def process_live_camera(self, show_preview=True):
        """
        Process live camera for real-time face recognition
        
        Args:
            show_preview: Whether to show preview
        """
        if not self.is_loaded:
            print("Error: Model and data not loaded!")
            return
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Could not open camera")
            return
        
        print("Starting live face recognition...")
        print("Press 'q' to quit")
        
        frame_count = 0
        recognition_results = []
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to read frame from camera")
                break
            
            # Get reference positions (use frame 0 as reference for live mode)
            ref_positions = self.get_reference_positions(0, tolerance=10)
            
            # Convert to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Skip this frame if no template image
            if self.template_image is None:
                frame_count += 1
                continue
            
            # Store all match results for selecting best match
            all_matches = []
            
            # Process each reference position
            for ref_pos in ref_positions:
                # Define search area around reference position
                search_scale = 2.0  # Larger search area for live mode
                search_w = int(ref_pos['width'] * search_scale)
                search_h = int(ref_pos['height'] * search_scale)
                
                # Calculate search area boundaries
                height, width = frame.shape[:2]
                search_x = max(0, ref_pos['center_x'] - search_w // 2)
                search_y = max(0, ref_pos['center_y'] - search_h // 2)
                search_x_end = min(width, search_x + search_w)
                search_y_end = min(height, search_y + search_h)
                
                # Adjust search area size
                actual_search_w = search_x_end - search_x
                actual_search_h = search_y_end - search_y
                
                if actual_search_w > 0 and actual_search_h > 0:
                    # Extract search region
                    search_region = gray[search_y:search_y_end, search_x:search_x_end]
                    
                    # Resize template to match reference position size
                    template_resized = cv2.resize(self.template_image, (ref_pos['width'], ref_pos['height']))
                    
                    # Check if search region is large enough
                    if search_region.shape[0] >= template_resized.shape[0] and search_region.shape[1] >= template_resized.shape[1]:
                        # Perform template matching
                        result = cv2.matchTemplate(search_region, template_resized, cv2.TM_CCOEFF_NORMED)
                        _, max_val, _, max_loc = cv2.minMaxLoc(result)
                        
                        # Convert to global coordinates
                        global_x = search_x + max_loc[0]
                        global_y = search_y + max_loc[1]
                        
                        # Store match result
                        match_info = {
                            'x': global_x,
                            'y': global_y,
                            'width': ref_pos['width'],
                            'height': ref_pos['height'],
                            'confidence': max_val,
                            'ref_frame_diff': ref_pos['frame_diff']
                        }
                        all_matches.append(match_info)
            
            # Select best match (highest confidence)
            if all_matches:
                best_match = max(all_matches, key=lambda x: x['confidence'])
                
                # Only process if confidence is above threshold
                if best_match['confidence'] > 0.3:  # Lower threshold for live mode
                    # Extract face image for recognition
                    face_img = frame[best_match['y']:best_match['y']+best_match['height'], 
                                    best_match['x']:best_match['x']+best_match['width']]
                    
                    # Extract features and recognize
                    features = self.extract_face_features(face_img)
                    person_id, person_name, recognition_confidence = self.recognize_face(features)
                    
                    # Draw result
                    color = (0, 255, 0) if person_name != "unknown" else (0, 0, 255)
                    cv2.rectangle(frame, (best_match['x'], best_match['y']), 
                                 (best_match['x'] + best_match['width'], best_match['y'] + best_match['height']), color, 2)
                    
                    # Add label
                    label = f"{person_name} ({recognition_confidence:.2f}) TM:{best_match['confidence']:.2f}"
                    cv2.putText(frame, label, (best_match['x'], best_match['y'] - 10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Show preview
            if show_preview:
                cv2.imshow('Live Face Recognition', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            frame_count += 1
        
        # Clean up resources
        cap.release()
        cv2.destroyAllWindows()
        
        print("\nLive recognition stopped!")
        return recognition_results

    def process_video(self, video_path, output_path=None, show_preview=True):
        """
        Process video for face recognition
        
        Args:
            video_path: Input video path
            output_path: Output video path (optional)
            show_preview: Whether to show preview
        """
        if not self.is_loaded:
            print("Error: Model and data not loaded!")
            return
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video {video_path}")
            return
        
        # Get video information
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"Processing video: {video_path}")
        print(f"Resolution: {width}x{height}, FPS: {fps}, Total frames: {total_frames}")
        
        # Setup output video
        out = None
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        recognition_results = []
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Get reference positions
            ref_positions = self.get_reference_positions(frame_count)
            
            # Convert to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Skip this frame if no template image
            if self.template_image is None:
                frame_count += 1
                continue
            
            # Store all match results for selecting best match
            all_matches = []
            
            # Process each reference position
            for ref_pos in ref_positions:
                # Define search area around reference position
                search_scale = 1.5  # Search area expansion factor
                search_w = int(ref_pos['width'] * search_scale)
                search_h = int(ref_pos['height'] * search_scale)
                
                # Calculate search area boundaries
                search_x = max(0, ref_pos['center_x'] - search_w // 2)
                search_y = max(0, ref_pos['center_y'] - search_h // 2)
                search_x_end = min(width, search_x + search_w)
                search_y_end = min(height, search_y + search_h)
                
                # Adjust search area size
                actual_search_w = search_x_end - search_x
                actual_search_h = search_y_end - search_y
                
                if actual_search_w > 0 and actual_search_h > 0:
                    # Extract search region
                    search_region = gray[search_y:search_y_end, search_x:search_x_end]
                    
                    # Resize template to match reference position size
                    template_resized = cv2.resize(self.template_image, (ref_pos['width'], ref_pos['height']))
                    
                    # Check if search region is large enough
                    if search_region.shape[0] >= template_resized.shape[0] and search_region.shape[1] >= template_resized.shape[1]:
                        # Perform template matching
                        result = cv2.matchTemplate(search_region, template_resized, cv2.TM_CCOEFF_NORMED)
                        _, max_val, _, max_loc = cv2.minMaxLoc(result)
                        
                        # Convert to global coordinates
                        global_x = search_x + max_loc[0]
                        global_y = search_y + max_loc[1]
                        
                        # Store match result
                        match_info = {
                            'x': global_x,
                            'y': global_y,
                            'width': ref_pos['width'],
                            'height': ref_pos['height'],
                            'confidence': max_val,
                            'ref_frame_diff': ref_pos['frame_diff']
                        }
                        all_matches.append(match_info)
            
            # Select best match (highest confidence)
            if all_matches:
                best_match = max(all_matches, key=lambda x: x['confidence'])
                
                # Extract face image for recognition
                face_img = frame[best_match['y']:best_match['y']+best_match['height'], 
                                best_match['x']:best_match['x']+best_match['width']]
                
                # Extract features and recognize
                features = self.extract_face_features(face_img)
                person_id, person_name, recognition_confidence = self.recognize_face(features)
                
                # Record result
                result = {
                    'frame_number': int(frame_count),
                    'timestamp': float(frame_count / fps if fps > 0 else 0),
                    'x': int(best_match['x']),
                    'y': int(best_match['y']),
                    'width': int(best_match['width']),
                    'height': int(best_match['height']),
                    'person_id': int(person_id),
                    'person_name': str(person_name),
                    'confidence': float(recognition_confidence),
                    'template_match_confidence': float(best_match['confidence']),
                    'ref_frame_diff': int(best_match['ref_frame_diff'])
                }
                recognition_results.append(result)
                
                # Draw result
                color = (0, 255, 0) if person_name != "unknown" else (0, 0, 255)
                cv2.rectangle(frame, (best_match['x'], best_match['y']), 
                                (best_match['x'] + best_match['width'], best_match['y'] + best_match['height']), color, 2)
                
                # Add label
                label = f"{person_name} ({recognition_confidence:.2f}) TM:{best_match['confidence']:.2f}"
                cv2.putText(frame, label, (best_match['x'], best_match['y'] - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Show preview
            if show_preview:
                cv2.imshow('Face Recognition', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            # Write to output video
            if out:
                out.write(frame)
            
            frame_count += 1
            
            # Show progress
            if frame_count % 100 == 0:
                progress = (frame_count / total_frames) * 100
                print(f"Progress: {progress:.1f}% ({frame_count}/{total_frames} frames)")
        
        # Clean up resources
        cap.release()
        if out:
            out.release()
        cv2.destroyAllWindows()
        
        # Save recognition results
        results_path = output_path.replace('recognition_output.mp4', 'recognition_results.json') if output_path else "recognition_results.json"
        results_data = {
            'video_path': video_path,
            'total_frames': total_frames,
            'fps': fps,
            'total_recognitions': len(recognition_results),
            'processing_date': datetime.now().isoformat(),
            'results': recognition_results
        }
        
        with open(results_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=2, ensure_ascii=False)
        
        print(f"\nRecognition completed!")
        print(f"Total recognitions: {len(recognition_results)}")
        print(f"Results saved to: {results_path}")
        if output_path:
            print(f"Output video saved to: {output_path}")
        
        return recognition_results

## User Input Section

Please provide the required information:

In [6]:
# Get user input
person_name = input("Enter person name: ")

# Choose input mode
print("\nChoose input mode:")
print("1. Video file")
print("2. Live camera")
mode_choice = input("Enter your choice (1 or 2): ")

is_live_mode = mode_choice == '2'
video_input = None

if not is_live_mode:
    video_input = input("Enter video file path: ")

print(f"Person name: {person_name}")
print(f"Mode: {'Live camera' if is_live_mode else 'Video file'}")
if video_input:
    print(f"Video input: {video_input}")


Choose input mode:
1. Video file
2. Live camera
Person name: Joseph_Lai
Mode: Video file
Video input: Joseph_Lai.mp4


In [7]:
# Configure paths based on person name
base_dir = f"faces/lock_version/{person_name}"
model_path = f"{base_dir}/face_model.pkl"
detection_json_path = f"{base_dir}/{person_name}_faces_detection.json"
output_video = f"{base_dir}/recognition_output.mp4"
video_input = f"videos/{video_input}"

print(f"Model path: {model_path}")
print(f"Detection JSON: {detection_json_path}")
print(f"Output video: {output_video}")
print(f"Video input: {video_input}")


Model path: faces/lock_version/Joseph_Lai/face_model.pkl
Detection JSON: faces/lock_version/Joseph_Lai/Joseph_Lai_faces_detection.json
Output video: faces/lock_version/Joseph_Lai/recognition_output.mp4
Video input: videos/Joseph_Lai.mp4


In [8]:
# Check necessary files
if not os.path.exists(model_path):
    print(f"Error: Model file {model_path} not found!")
    print("Please run train-v2.py first to train the model.")
else:
    print("Model file found.")

if not os.path.exists(detection_json_path):
    print(f"Error: Detection JSON {detection_json_path} not found!")
    print("Please run detection-v2.py first to generate detection data.")
else:
    print("Detection JSON found.")

if not is_live_mode and video_input and not os.path.exists(video_input):
    print(f"Error: Video file {video_input} not found!")
elif not is_live_mode and video_input:
    print("Video file found.")

Model file found.
Detection JSON found.
Video file found.


In [16]:
# Create scanner and load model
scanner = FaceScanner(model_path, detection_json_path)

# Load model and data
if scanner.load_model_and_data():
    print("Model and data loaded successfully!")
else:
    print("Failed to load model and data!")

PCA model loaded: 77 faces, 1 persons
Detection data loaded: 77 detected faces
Template image loaded: faces/lock_version/Joseph_Lai\face_000000_frame_000000.jpg
Model and data loaded successfully!


In [17]:
# Process video or live camera
if scanner.is_loaded:
    if is_live_mode:
        print("\nStarting live face recognition...")
        print("Press 'q' to quit")
        results = scanner.process_live_camera(show_preview=True)
    else:
        if video_input and os.path.exists(video_input):
            print("\nStarting face recognition...")
            print("Press 'q' to quit preview window")
            results = scanner.process_video(video_input, output_video, show_preview=True)
            
            if results:
                print(f"\nRecognition summary:")
                # Count recognition results
                person_counts = {}
                for result in results:
                    name = result['person_name']
                    person_counts[name] = person_counts.get(name, 0) + 1
                
                for name, count in person_counts.items():
                    print(f"  {name}: {count} detections")
        else:
            print("Video file not specified or not found!")
else:
    print("Scanner not loaded properly!")


Starting face recognition...
Press 'q' to quit preview window
Processing video: videos/Joseph_Lai.mp4
Resolution: 480x848, FPS: 30.0, Total frames: 131


Progress: 76.3% (100/131 frames)

Recognition completed!
Total recognitions: 109
Results saved to: faces/lock_version/Joseph_Lai/recognition_results.json
Output video saved to: faces/lock_version/Joseph_Lai/recognition_output.mp4

Recognition summary:
  Joseph_Lai: 87 detections
  unknown: 22 detections
